In [ ]:
!pip install -r requirements.txt

  Cloning https://github.com/huggingface/peft to /tmp/pip-req-build-9vinhczr
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-9vinhczr
  Resolved https://github.com/huggingface/peft to commit 02b5aeddf9c1ea11451f10a8a26da7e5df8cca4a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/trl.git to /tmp/pip-req-build-80u6phej
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-80u6phej
  Resolved https://github.com/huggingface/trl.git to commit ab0d11d81550618546cd1d8807627a594a58f029
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8

### Imports

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import load_dataset

### Quantization

In [ ]:
# https://huggingface.co/docs/peft/main/en/developer_guides/quantization

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
# Load in the model
model_id = "nvidia/OpenMath-Mistral-7B-v0.1-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
quantized_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0", quantization_config=quantization_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token
tokenizer.padding_side = "right"

In [ ]:
model = prepare_model_for_kbit_training(quantized_model)

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = get_peft_model(model, lora_config)

### Datahandling

In [ ]:
from datasets import load_dataset

In [ ]:
train_dataset = load_dataset('json', data_files="/content/merged_math_problems_train.json", split="train")
val_dataset = load_dataset('json', data_files="/content/merged_math_problems_test.json")['train']

In [ ]:
train_dataset

Dataset({
    features: ['level', 'solution', 'type', 'problem'],
    num_rows: 7500
})

In [ ]:
val_dataset

5000

In [ ]:
text_format = """### Question: {problem_statement} [Answer] : {solution}"""

def formatting_prompts_func(dataset):
    output_texts = []

    for idx in range(len(dataset['level'])):
        problem = dataset['problem'][idx]
        solution = dataset['solution'][idx]
        text = text_format.format(problem_statement=problem, solution=solution)
        output_texts.append(text)

    return output_texts

In [ ]:
response_template = " [Answer] :"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [ ]:
trainer = SFTTrainer(
    model,
    train_dataset=train_dataset,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:141: UserWarning: Could not find response key ` [Answer] :` in the following instance: <s> ### Question: The Cubs are playing the Red Sox in the World Series. To win the world series, a team must win 4 games before the other team does. If the Cubs win each game with probability $\dfrac{3}{5}$ and there are no ties, what is the probability that the Cubs will win the World Series? Express your answer as a percent rounded to the nearest whole percent. [Answer] : There are four possible cases for the Cubs winning the World Series, depending on the number of games that Red Sox win before the Cubs win their fourth game: the Red Sox can win no games, one game, two games, or three games. In general, if the Red Sox win exactly $k$ games before the Cubs win their 4th game, there will be a total of $3+k$ games played before the last one (which the Cubs must win), there will be a total of $\dbinom{3+k}{k}$ ways of selecting the games tha

KeyboardInterrupt: 